In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
import RigidBodyDynamics.Spatial
import LCPSim
import CartPoles
import Nets
using JuMP
using Gurobi
using ProfileView
import ConditionalJuMP
using BenchmarkTools
using StaticArrays
using Base.Test

In [ ]:
cartpole = CartPoles.CartPole()
mechanism = cartpole.mechanism
x0 = MechanismState{Float64}(mechanism)

In [ ]:
x = LCPSim.StateRecord(x0)
xnext = LCPSim.LinearizedState{Variable}(x0)
u = zeros(2)
model = Model(solver=GurobiSolver())
@benchmark LCPSim.update($x, $xnext, $u, $(cartpole.environment), 0.04, $model)

In [ ]:
m = Model()
@variable m -1 <= q[1:4] <= 1
@variable m -1 <= y <= 1
d = randn(4)' * q
@benchmark ConditionalJuMP.@disjunction($m, ($d == 0), ($y == 0))

In [ ]:
x = LCPSim.StateRecord(x0)
xnext = LCPSim.LinearizedState{Variable}(x0)
u = zeros(2)
model = Model(solver=GurobiSolver())
f() = LCPSim.optimize(x0, cartpole.environment, 0.04, 100, model)
# f() = for i in 1:100; LCPSim.update(x, xnext, u, cartpole.environment, 0.04, model); end

f()
Profile.clear()
@time @profile f();
ProfileView.view()

In [ ]:
x0 = MechanismState{Float64}(mechanism)
g() = LCPSim.simulate(x0, x -> zeros(num_velocities(x0)), cartpole.environment, 0.04, 1000, GurobiSolver(Gurobi.Env(), OutputFlag=0))
g()
Profile.clear()
@time @profile g()
ProfileView.view()

In [ ]:
cj = ConditionalJuMP

In [ ]:
m = Model()
@variable m -1 <= q[1:10] <= 1
y = randn(length(q))
v = [randn(length(q))' * q for _ in q];

In [ ]:
cj.@implies(m, q[1] <= 0 => q[2] == 2)

In [ ]:
im = cj.getindmap!(m)

In [ ]:
im.disjunctions

In [ ]:
ex = rand(100)' * rand(q, 100)

ex.vars

In [ ]:
@benchmark ConditionalJuMP.simplify($ex)

In [ ]:
@benchmark ConditionalJuMP.simplify!(copy($ex))

In [ ]:
Profile.clear()
@profile for i in 1:100000; ConditionalJuMP.simplify(ex); end
ProfileView.view()

In [ ]:
@benchmark dot($y, $v)

In [ ]:
m = Model()
@variable m -1 <= x[1:10] <= 1
y = sum(x)

In [ ]:
g() = for i in 1:10000; ConditionalJuMP.upperbound(y); end
g()
Profile.clear()
@profile @time g()
ProfileView.view()

In [ ]:
@benchmark ConditionalJuMP.upperbound($y)

In [ ]:
@code_warntype LCPSim.update(x, xnext, u, cartpole.environment, 0.04, model)